In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')

In [ ]:
df=pd.read_csv("spam_ham_dataset.csv")

In [ ]:
# df  # Data loaded


In [ ]:
df=df.drop(columns=["Unnamed: 0","label_num"])

In [ ]:
# df  # Data loaded


In [ ]:
df['spam']=np.where(df['label']=='spam', 1,0)

In [ ]:
# df  # Data loaded


In [ ]:
    df.isnull().sum()

In [ ]:
# df=df.dropna()
# df=df.drop_duplicates()

In [ ]:
# print(f"Duplicated values in a dataset: {df.duplicated().sum()}")

In [ ]:
def clean_text(text):

    text=''.join([char for char in text if char not in string.punctuation])
    words=text.split()
    stop_words=stopwords.words("english")
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]
    return ' '.join(words)


df['clean_text'] = df['text'].apply(clean_text)
    

In [ ]:
# df.head(10)['clean_text']

In [ ]:
df=df.drop(columns=['text'])

In [ ]:
# df  # Data loaded


In [ ]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(df["clean_text"])
y=df["spam"]

In [ ]:
X_train, X_text, y_train, y_test=train_test_split(X, y ,test_size=0.2,random_state=42)

In [ ]:
# Define models
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(),
}

best_results = {}
best_model_name = None
best_accuracy = 0.0 

# Train and evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)  # Training  the model
    y_pred = model.predict(X_text)  # Predict on the test set

#     accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    best_results[name] = accuracy  # Store accuracy in the results dictionary

    # Display accuracy and classification report for the current model
#     print(f"{name} Accuracy: {accuracy:.2f}")
#     print("-----------------------------------------------------")
#     print(classification_report(y_test, y_pred))

    # Check if this is the best accuracy so far
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = name

# Display the model with the best accuracy at the end
# print("\nBest Model:")
# print(f"{best_model_name} with Accuracy: {best_accuracy:.2f}")

In [ ]:
data = {
    'spam': ['Congratulations! You’ve won a $1,000 gift card. Click here to claim your prize']
}

In [ ]:
db1=pd.DataFrame(data)


In [ ]:
r=vectorizer.transform(db1["spam"])

In [ ]:
 ruc = model.predict(r)

In [ ]:
# Sample prediction results
# for email, pred in zip(db1['spam'], ruc):
#     print(email, "->", "Spam" if pred==1 else "Not Spam")


In [ ]:
# Interactive UI for Email Spam Detection
import ipywidgets as widgets
from IPython.display import display

# Create widgets
email_input = widgets.Textarea(
    value='',
    placeholder='Enter email content here to check if it\'s spam or not...',
    description='Email Text:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

predict_button = widgets.Button(
    description='🔍 Check Spam',
    button_style='primary',
    layout=widgets.Layout(width='200px', margin='10px 0px')
)

result_output = widgets.HTML(
    value='<div style="padding: 10px; text-align: center; color: #666;">Enter an email above and click "Check Spam" to see the prediction</div>',
    layout=widgets.Layout(width='100%', margin='10px 0px')
)

def predict_spam(b):
    try:
        email_text = email_input.value.strip()
        
        if not email_text:
            result_output.value = '<div style="padding: 15px; background-color: #fff3cd; border: 1px solid #ffeaa7; border-radius: 5px; color: #856404;"><strong>⚠️ Warning:</strong> Please enter some email content to analyze.</div>'
            return
        
        # Clean the text using the same function
        cleaned_email = clean_text(email_text)
        
        # Transform using the vectorizer
        email_vectorized = vectorizer.transform([cleaned_email])
        
        # Make prediction using Naive Bayes (best model)
        prediction = models['Naive Bayes'].predict(email_vectorized)[0]
        
        # Display result
        if prediction == 1:
            result_output.value = '''
            <div style="padding: 20px; background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 10px; text-align: center;">
                <h3 style="color: #721c24; margin: 0 0 10px 0;">🚨 SPAM DETECTED</h3>
                <p style="margin: 0; font-size: 16px; color: #721c24;">This email appears to be spam.</p>
                <p style="margin: 10px 0 0 0; font-size: 14px; color: #856404;">Model: Naive Bayes (97% accuracy)</p>
            </div>
            '''
        else:
            result_output.value = '''
            <div style="padding: 20px; background-color: #d4edda; border: 1px solid #c3e6cb; border-radius: 10px; text-align: center;">
                <h3 style="color: #155724; margin: 0 0 10px 0;">✅ NOT SPAM</h3>
                <p style="margin: 0; font-size: 16px; color: #155724;">This email appears to be legitimate.</p>
                <p style="margin: 10px 0 0 0; font-size: 14px; color: #856404;">Model: Naive Bayes (97% accuracy)</p>
            </div>
            '''
        
    except Exception as e:
        result_output.value = f'<div style="padding: 15px; background-color: #f8d7da; border: 1px solid #f5c6cb; border-radius: 5px; color: #721c24;"><strong>❌ Error:</strong> {str(e)}</div>'

# Connect button
predict_button.on_click(predict_spam)

# Display UI
ui_container = widgets.VBox([
    widgets.HTML('<h2 style="text-align: center; color: #333; margin-bottom: 20px;">📧 Email Spam Detector</h2>'),
    widgets.HTML('<p style="text-align: center; color: #666; margin-bottom: 20px;">Powered by Machine Learning - 97% Accuracy</p>'),
    email_input,
    predict_button,
    result_output
], layout=widgets.Layout(padding='20px', border='1px solid #ddd', border_radius='10px', margin='20px'))

display(ui_container)